In [2]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Project 3 | Matrix Factorization Methods 

## Jester Joke Data
2.5 million anonymous ratings of jokes by users of the Jester Joke Recommender System (Ken Goldberg, AUTOLab, UC Berkeley).  Values from (-10.00 to +10.00) of 100 jokes collected between April 1999 - May 2003.  Data from 24,983 users who have rated 36 or more jokes, a matrix with dimensions 24983 X 101. 

In [4]:
#import joke data
jokes_df = pd.read_csv('jester-data-1.csv')

### Sparseness -- Stochastic Gradient Descent
This dataset is relatively sparse. There are alot of jokes that users did not rate. High portion of missing values caused by sparseness in the user-item ratings matrix. Carelessly addressing only the relatively few known entries is highly prone to overfitting. Previously, I had used a dense subcluster to approximate user similarity. 

SVD requires that there are no missing values. This time i will use stochastic gradient descent (SGD) to prepare the data for matrix factorization. SGD loops through ratings in the training set. For each given training case, the system predicts rui and computes the associated prediction error portional to g in the opposite direction of the gradient.  Instead of using the full dataset to compute the gradient at each step, SGD uses only one random data point (or a small batch of data points) at each iteration. This makes the computation much faster.

In [5]:
def sgd(X, y, learning_rate=0.1, epochs=1000, batch_size=1):
    m = len(X)  
    theta = np.random.randn(2, 1) 
    
    X_bias = np.c_[np.ones((m, 1)), X]

    cost_history = []  

    for epoch in range(epochs):
        indices = np.random.permutation(m)
        X_shuffled = X_bias[indices]
        y_shuffled = y[indices]

        for i in range(0, m, batch_size):
            X_batch = X_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]

            gradients = 2 / batch_size * X_batch.T.dot(X_batch.dot(theta) - y_batch)
            theta -= learning_rate * gradients

        predictions = X_bias.dot(theta)
        cost = np.mean((predictions - y) ** 2)
        cost_history.append(cost)
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Cost: {cost}")

    return theta, cost_history

In [ ]:
theta_final, cost_history = sgd(X, y, learning_rate=0.1, epochs=1000, batch_size=1)

### Visualizing the Cost Function:

In [ ]:
plt.plot(cost_history)
plt.xlabel('Epochs')
plt.ylabel('Cost (MSE)')
plt.title('Cost Function during Training')
plt.show()

### Plotting the Data and Regression Line

In [ ]:
plt.scatter(X, y, color='blue', label='Data points')
plt.plot(X, np.c_[np.ones((X.shape[0], 1)), X].dot(theta_final), color='red', label='SGD fit line')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression using Stochastic Gradient Descent')
plt.legend()
plt.show()

### Building a Complete UV-Decomposition Algorithm (Method: Gradient Descent)
		1. Preprocessing of the matrix M.
			i. Subtract the average rating of the user, and the average rating of the item
			ii. We have to undo the normalization at the end. Add back the above
		2. Initializing U and V 
			i. Simple starting is to give each element of U and V the same value. (probably 0 after normalization)
		3. Ordering the optimization of the elements of U and V 
			i. Simplest is row-by-row, and visit them round robin fashion
			ii. Note that just because we optimized an element once does not mean we cannot find a better value for that element after other elements have been adjusted. Thus, we need to visit elements repeatedly
		4. Ending the attempt at optimization.
			i. Ideally, at some point the RMSE becomes 0, and we know we cannot do better.
			ii. In practice, since there are normally many more nonblank elements in M than there are elements in U and V together, we have no right to expect that we can reduce the RMSE to 0
We can track the amount of improvement in the RMSE obtained in one round of the optimization, and stop when that improvement falls below a threshold.

## Matrix factorization models 
are superior to classic nearest-neighbor techniques for producing product recommendations, allowing the incorporation of additional information such as implicit feedback, temporal effects, and confidence levels.	- Matrix factorization:
models map both users and items to a joint latent factor space of dimensionality f, such that user-item interactions are modeled as inner products in that space.
Storage reduction can be meaningful with large datasets, makes your matrixes smaller , without making your error too much higher:


## Preprocessing: 
SVD can be thought of as a pre-processing step for feature engineering. You might easily start 
with thousands or millions of items, and use SVD to create a much smaller set of “k” items 
(e.g. 20 or 70).

## Reduce dimensionality:
by applying SVD to transform a large user-item matrix into a smallerset of latent features, simplifying the data while preserving its essential patterns.

## Evaluate the effectiveness of the matrix factorization model:
by assessing the quality of predictions using performance metrics such as RMSE, and comparing these results with those from
baseline models.  

Root-Mean-Square Error
		○ While we can pick among several measures of how close the product UV is to M, the typical choice is the root-mean-square error (RMSE), where we
			i. Sum, over all nonblank entries in M the square of the difference between that entry and the corresponding entry in the product UV .
			ii. Take the mean (average) of these squares by dividing by the number of terms in the sum (i.e., the number of nonblank entries in M).
			iii. Take the square root of the mean

	- Local Minima:
		○ matrices U and V such that no allowable adjustment reduces the RMSE
		○ Global Minimum – Unfortunately, only one of these local minima will be the matrices U and V that produce the least possible RMSE
There is never a guarantee that our best local minimum will be the global minimum.

## Overfitting: 
although the RMSE may be small on the given data, it doesn’t do well predicting future data
		○ UV-decomposition is that we arrive at one of the many local minima that conform well to the given data, but picks up values in the data that don’t reflect well the underlying process that gives rise to the data
		
	- Avoiding Overfitting:
			1. Avoid favoring the first components to be optimized by only moving the value of a component a fraction of the way, say half way, from its current value toward its optimized value.
			2. Stop revisiting elements of U and V well before the process has converged.
Take several different UV decompositions, and when predicting a new entry in the matrix M, take the average of the results of using each decomposition.

## Understand the limitations and trade-offs of matrix factorization techniques, 
including the computational cost of SVD, the lack of explainability, and the challenges in handling sparse
datasets with many missing values.

## Implement optimizations to improve the scalability and efficiency of matrix factorization models,
including using subsets of data, parallelization, or other advanced techniques to manage memory and computation costs.